In [1]:
from collections import defaultdict
import datetime
from gensim import corpora, models, similarities
from matplotlib import pyplot as plt
import numpy as np
import operator
import pandas as pd
import pickle
import random
import re
import scipy
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier

from tqdm import tqdm_notebook

import src.knntools as knntools
import src.postprocess as postprocess
import src.preprocess as preprocess
import src.tfidftools as tfidftools
import src.tools as tools
import src.recencytools as recency
import src.scoring as scoring
import src.textembeddingtools as texttools

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
path_to_data = 'data/'

training = pd.read_csv(path_to_data + 'training_set.csv', sep=',', header=0)

training_info = pd.read_csv(
    path_to_data + 'training_info.csv', sep=',', parse_dates=True, header=0)

test = pd.read_csv(path_to_data + 'test_set.csv', sep=',', header=0)

test_info = pd.read_csv(path_to_data + 'test_info.csv',
                        sep=',', parse_dates=True, header=0)
path_to_results = 'results/'

In [30]:
train_info, train_email_ids_per_sender, val_info, val_email_ids_per_sender = scoring.get_train_val(training, training_info, train_frac=0.95)

Processing training !

Processing val !



In [ ]:
train_body_dict = preprocess.body_dict_from_panda(train_info)
val_body_dict = preprocess.body_dict_from_panda(val_info)


In [ ]:
train_token_dict = texttools.get_token_dict(train_body_dict)
val_token_dict = texttools.get_token_dict(val_body_dict)

34303/|/ 83%|| 34303/41432 [02:03<00:25, 278.86it/s]

In [ ]:
train_email_list, train_idx_to_mids = texttools.token_dicts_to_token_lists(train_token_dict, rarity_threshold=3)
val_email_list, val_idx_to_mids = texttools.token_dicts_to_token_lists(val_token_dict, rarity_threshold=3)

In [7]:
# Assign unique id to word
train_word_id_dic = corpora.Dictionary(train_email_list)

# Compute email corpush as bow [[(wordid_1_1, count_1_1), ...] ...]
train_email_corpus = [train_word_id_dic.doc2bow(text) for text in train_email_list]
val_email_corpus = [train_word_id_dic.doc2bow(text) for text in val_email_list]

In [8]:
nb_topics = 200
nb_emails = len(train_email_corpus)
print(nb_emails)

41432


In [10]:
# model_vars_path = 'variables/hdp_similarities_{thres_nb}_words_out_no_punct'.format(thres_nb=rarity_thres)
stored_model_results = ['variables/train-lda-200.p']

lda_model = texttools.compute_model(train_email_corpus, train_word_id_dic, model='lda', nb_topics=nb_topics,
                                    use_saved=True, save=True, model_results_path=stored_model_results[0])

In [12]:
train_lda_vectors = lda_model[train_email_corpus]
val_lda_vectors = lda_model[val_email_corpus]

In [14]:
len(train_lda_vectors)

41432

In [61]:
# Takes 5 minutes ! Consider loading saved file
train_lda_feature_matrix = texttools.model_vectors_to_matrix_features(train_lda_vectors, nb_topics)

In [15]:
train_lda_features_path = 'variables/train-lda-200-features.p'

# with open(train_lda_features_path, 'wb') as outfile:
#     pickle.dump(train_lda_feature_matrix, outfile)
    
with open(train_lda_features_path, 'rb') as infile:
    train_lda_feature_matrix = pickle.load(infile)

In [16]:
val_lda_feature_matrix = texttools.model_vectors_to_matrix_features(val_lda_vectors, nb_topics)

In [17]:
lda_model.print_topics(2,10)

[(172,
  '0.078*"play" + 0.053*"julia" + 0.047*"dallas" + 0.043*"team" + 0.030*"db" + 0.028*"carson" + 0.026*"games" + 0.018*"occurs" + 0.017*"lamar" + 0.016*"fans"'),
 (33,
  '0.055*"duke" + 0.048*"currency" + 0.046*"pipeline" + 0.034*"south" + 0.030*"gas" + 0.028*"river" + 0.018*"christmas" + 0.015*"forwarding" + 0.014*"project" + 0.013*"usd"')]

In [18]:
train_day_features = recency.get_sparse_date_info(train_idx_to_mids, train_info)
val_day_features = recency.get_sparse_date_info(val_idx_to_mids, val_info)

In [19]:
print(train_day_features.shape)
print(val_day_features.shape)

(41432, 7)
(2181, 7)


In [20]:
train_sender_idx_dic = preprocess.get_sender_idx_dics(train_email_ids_per_sender)
train_sender_features = preprocess.get_sparse_sender_info(train_idx_to_mids, train_sender_idx_dic,
                                       train_email_ids_per_sender, train_info)
val_sender_features = preprocess.get_sparse_sender_info(val_idx_to_mids, train_sender_idx_dic,
                                       val_email_ids_per_sender, val_info)

In [21]:
print(train_sender_features.shape)
print(val_sender_features.shape)
print(np.sum(train_sender_features))
print(np.sum(val_sender_features))


(41432, 125)
(2181, 125)
41432.0
2181.0


In [22]:
stacked_train_features = scipy.sparse.hstack([train_sender_features, train_day_features, train_lda_feature_matrix])

In [23]:
stacked_val_features = scipy.sparse.hstack([val_sender_features, val_day_features, val_lda_feature_matrix])

In [24]:
train_predictions = preprocess.get_ordered_recipients(train_info, train_idx_to_mids)

In [25]:
binarized_train_predictions = MultiLabelBinarizer().fit_transform(train_predictions)

In [26]:
print(binarized_train_predictions.shape)

(41432, 9597)


In [28]:
rdm_train = RandomForestClassifier()

In [29]:
%%time
rdm_train.fit(stacked_train_features, binarized_train_predictions)

MemoryError: 

In [ ]:
rdm_train.predict(stacked_val_features)